In this part of the tutorial, we'll create materialized views: precomputed queries stored as physical tables. Some reasons you might want materialized views:

- Training on data only from a certain country
- Building different models on shirts, pants, dresses, and coats
- Using only a subset of data to test your training pipeline, then swapping in the whole dataset later

LanceDB makes materialized views super easy to create and keep updated; let's jump in!

In [ ]:
!pip install --upgrade geneva pandas "ray[default]" kubernetes

## 1. Data Ingestion

We'll use the same fashion-product-images dataset as in the Feature Engineering 101 tutorial. All the data loading is the same as that tutorial, but we'll also grab the `articleType`, and we'll make **one important change** while creating the Lance table:

## Enable Stable Row IDs

In classic Lance tables, compaction may cause row IDs to change. This causes problems for materialized views, which are based on the row IDs. So, when creating our table, we will use the `new_table_enable_stable_row_ids` option:

    
    db.create_table(
        name='my_table',
        data=my_data,
        storage_options={'new_table_enable_stable_row_ids': 'true'}
    )

In [ ]:
#!sudo rm -r db fashion-dataset # Uncomment and run this to delete the dataset if it already exists

# Download the dataset if it doesn't exist
!test -d fashion-dataset && test -n "$(ls -A fashion-dataset 2>/dev/null)" && \
  echo "Dataset already exists, skipping download" || \
    (curl -L -o fashion-product-images-small.zip https://www.kaggle.com/api/v1/datasets/download/paramaggarwal/fashion-product-images-small \
    && unzip -q fashion-product-images-small.zip -d fashion-dataset/)

In [ ]:
import geneva
import pandas as pd
from pathlib import Path

DATASET_SIZE = 1000
CONCURRENCY = 4 # Increase this if you have more CPUs available and want it to run faster.
CHECKPOINT_SIZE = 300

IMG_DIR = Path("fashion-dataset/images")
STYLE_CSV = Path("fashion-dataset/styles.csv")
DB_PATH = "./db"
TABLE_NAME = "products"
INSERT_FRAG_SIZE = min(1000, DATASET_SIZE / 2)

In [ ]:
df = pd.read_csv(STYLE_CSV, on_bad_lines='skip')
df = df.dropna(subset=["id", "productDisplayName"])
df = df.drop_duplicates(subset=["id"], keep="first")

def generate_rows(df, img_dir, start = 0, end = DATASET_SIZE):
    for _, row in df.iloc[start:end].iterrows():
        img_path = img_dir / f"{row['id']}.jpg"
        if not img_path.exists():
            continue
        with open(img_path, "rb") as f:
            yield {
                "id": int(row["id"]),
                "description": row["productDisplayName"],
                "articleType": row["articleType"],
                "color": row["baseColour"],
                "image_bytes": f.read()
            }

db = geneva.connect(DB_PATH)

# Drop the table if it already exists so we can recreate it
try:
    table = db.drop_table(TABLE_NAME)
except ValueError as e:
    pass
    
data_stream = generate_rows(df, IMG_DIR)
table = None

rows = []
for row in data_stream:
    rows.append(row)
    if len(rows) == INSERT_FRAG_SIZE:
        if table:
            table.add(rows)
        else:
            # Important! Make sure to set "new_table_enable_stable_row_ids" to "true" to use materialized views!
            table = db.create_table(TABLE_NAME, data=rows, storage_options={'new_table_enable_stable_row_ids': 'true'})
        rows = []
if rows:
    table.add(rows)
    
len(table)

In [ ]:
table.to_pandas()

## 2. Materialized Views

Imagine that we work at a clothing company, and `table` represents all the styles of clothing that we sell. Then let's say that we're building a model that focuses on shirts, so we only care about shirts. We could run a query like this every time:

In [ ]:
table.search().where("articleType = 'Shirts'").to_pandas()

But this is unwieldy, and could become expensive if the original table got too big. Instead, we'll create a materialized view so we only have what we need. First we create the view, then we refresh it so it has current data, like so:

In [ ]:
try:
    import ray
    ray.shutdown()
    db.drop_table("shirts")
except ValueError as e:
    pass
shirts = table.search().where("articleType = 'Shirts'").create_materialized_view(db, "shirts")
with db.local_ray_context():
    shirts.refresh()
shirts.to_pandas()


Like in the 101 demo, we are using `db.local_ray_context()` to say "run on a Ray instance on this computer." It will be much slower than running on multiple remote machines, but will simplify setup for this tutorial.

### Updating our materialized view

The real value of materialized views is the ability to "view" the source table - to reflect changes in it! Let's assume that our source table had 500 more clothing items added to it:

In [ ]:
table.add(list(generate_rows(df, IMG_DIR, start = DATASET_SIZE, end = DATASET_SIZE + 500)))
table.to_pandas()

Now to see all the new Shirts reflected in the `shirts` table, we can simply `refresh()` our materialized view. This will pick up all the new Shirts from the original `table`. Importantly, it will only process these 500 new rows! For our demo, that's not a big difference, but imagine a table with 10 years' worth of data that adds new data every day: only processing the new day's data will take about 0.02% of the time compared to reprocessing the whole table.

In [ ]:
with db.local_ray_context():
    shirts.refresh()
shirts.to_pandas()

### Custom columns

You can customize your materialized views by renaming columns and using sql expressions ([Datafusion dialect](https://datafusion.apache.org/user-guide/sql/index.html)) to create new columns. Say we want to find all Jeans, but our department uses "clothingType" instead of "articleType", and we want a simple flag to separate out blue jeans vs other jeans:

In [ ]:
try:
    db.drop_table("jeans")
except ValueError as e:
    pass
jeans = table.search().where("articleType = 'Jeans'")\
    .select({"description": "description",
             "color": "color",
             "clothingType": "articleType",
             "image": "image_bytes",
             "isBlue": "color == 'Blue'"})\
    .create_materialized_view(db, "jeans")
with db.local_ray_context():
    jeans.refresh()
jeans.to_pandas()

## Wrapping up

This short demo shows the powerful views you can create very simply with LanceDB Materialized Views. For more, check out our [docs](https://docs.lancedb.com/geneva/jobs/materialized-views). In the next section, we'll go back to working with backfills, focusing on connecting to remote Ray clusters to do feature engineering at production scale.